In [52]:
# Dependencies
import pandas as pd
import requests
import json

# Google developer API key
from config import gkey

In [53]:
# Import csv files as DataFrames
Aliens1_df = pd.read_csv("Data/scrubbed.csv")
Aliens1_df.head()

C:\Users\Doggger\anaconda3\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (5,9) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude
0,10/10/1949 20:30,san marcos,tx,us,cylinder,2700,45 minutes,This event took place in early fall around 194...,4/27/2004,29.8830556,-97.941111
1,10/10/1949 21:00,lackland afb,tx,NaN,light,7200,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,12/16/2005,29.38421,-98.581082
2,10/10/1955 17:00,chester (uk/england),NaN,gb,circle,20,20 seconds,Green/Orange circular disc over Chester&#44 En...,1/21/2008,53.2,-2.916667
3,10/10/1956 21:00,edna,tx,us,circle,20,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.9783333,-96.645833
4,10/10/1960 20:00,kaneohe,hi,us,light,900,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.4180556,-157.803611


In [58]:
# Selecting subset of US
Aliens1_df = Aliens1_df.loc[(Aliens1_df['country'] == 'us')]

#Creating new vairables for month, day, time and year from datetime
Aliens1_df[['Month', 'Day', 'Time']] = Aliens1_df['datetime'].str.split('/',expand=True)
Aliens1_df['Year']= Aliens1_df['Time'].str[0:4]
Aliens1_df['Time']= Aliens1_df['Time'].str[5:10]

#Further selecting subset for time frame on interest
Aliens1_df = Aliens1_df[Aliens1_df['Year'].between('1993', '2020')]

#Removes the space in the longitude column name
Aliens1_df = Aliens1_df.rename(columns={'longitude ': 'longitude'})
Aliens1_df.head()

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude,Month,Day,Time,Year
56,10/10/1993 22:00,peoria,il,us,light,8,8 seconds,Light over Peoria&#44 IL that moves slowly&#44...,10/11/2005,40.6936111,-89.588889,10,10,22:00,1993
57,10/10/1993 23:00,carthage,tn,us,other,60,less than 1 min,1 object with green and red lights,3/21/2003,36.2522222,-85.951667,10,10,23:00,1993
59,10/10/1994 18:30,burnt hills,ny,us,rectangle,120,2 minutes,Giant rectangular craft (resembling an upside-...,10/23/2013,42.9097222,-73.895556,10,10,18:30,1994
60,10/10/1994 22:00,pinebergen,ar,us,light,5,5 seconds,Round&#44 bright&#44 low flying object silentl...,2/18/2001,34.1027778,-91.992222,10,10,22:00,1994
62,10/10/1995 19:45,milwaukee,wi,us,NaN,120,2 min.,Man on Hwy 43 SW of Milwaukee sees large&#44 ...,11/2/1999,43.0388889,-87.906389,10,10,19:45,1995


In [69]:
# Creates new df with only 2013 data
Aliens2013_df = Aliens1_df.loc[(Aliens1_df['Year'] == '2013')]
Aliens2013_df.head()

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude,Month,Day,Time,Year
267,10/10/2013 02:32,palm harbor,fl,us,chevron,5,5 seconds,Tracking north to south at apx 2500 ft. A very...,10/14/2013,28.0777778,-82.763889,10,10,02:32,2013
269,10/10/2013 17:00,st. louis county,mo,us,light,10800,2-3 hours,Hovering bright object moving slowly around St...,10/23/2013,38.6272222,-90.197778,10,10,17:00,2013
271,10/10/2013 19:00,rittman,oh,us,light,60,1 minute,Fast moving white light.,10/14/2013,40.9780556,-81.782222,10,10,19:00,2013
273,10/10/2013 19:20,essex junction,vt,us,light,120,2 minutes,Noiseless&#44 low flying&#44 white&#44 bright ...,10/14/2013,44.4905556,-73.111389,10,10,19:20,2013
274,10/10/2013 19:20,kenner,la,us,circle,600,10 minutes,2 circle lights moving erratically over Kenner...,10/14/2013,29.9938889,-90.241667,10,10,19:20,2013


In [ ]:
Aliens2013_df['county'] = ""

params = {"key": gkey}
params['result_type'] = 'administrative_area_level_2'
 
for index, row in Aliens2013_df.head(1).iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"

    lat = row['latitude']
    lng = row['longitude']

    # update address key value
    params['latlng'] = f"{lat},{lng}"

    # make request and convert to json
    counties_df = requests.get(base_url, params=params).json()
    
    print(json.dumps(counties_df, indent=4, sort_keys=True))
    
    #Aliens2013_df.loc[index, 'county'] = counties_df["results"][0]["address_components"][0]["long_name"]

#Aliens2013_df.head()

In [ ]:
states = {
        'AK': 'O',
        'AL': 'S',
        'AR': 'S',
        'AS': 'O',
        'AZ': 'W',
        'CA': 'W',
        'CO': 'W',
        'CT': 'N',
        'DC': 'N',
        'DE': 'N',
        'FL': 'S',
        'GA': 'S',
        'GU': 'O',
        'HI': 'O',
        'IA': 'M',
        'ID': 'W',
        'IL': 'M',
        'IN': 'M',
        'KS': 'M',
        'KY': 'S',
        'LA': 'S',
        'MA': 'N',
        'MD': 'N',
        'ME': 'N',
        'MI': 'W',
        'MN': 'M',
        'MO': 'M',
        'MP': 'O',
        'MS': 'S',
        'MT': 'W',
        'NA': 'O',
        'NC': 'S',
        'ND': 'M',
        'NE': 'W',
        'NH': 'N',
        'NJ': 'N',
        'NM': 'W',
        'NV': 'W',
        'NY': 'N',
        'OH': 'M',
        'OK': 'S',
        'OR': 'W',
        'PA': 'N',
        'PR': 'O',
        'RI': 'N',
        'SC': 'S',
        'SD': 'M',
        'TN': 'S',
        'TX': 'S',
        'UT': 'W',
        'VA': 'S',
        'VI': 'O',
        'VT': 'N',
        'WA': 'W',
        'WI': 'M',
        'WV': 'S',
        'WY': 'W'
}